This piece of code tests how VQE works with different classical optimization routines.

In [3]:
# Importing packages neeeded to define the molecule and map the hamiltonian

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

# First we define a function that takes as input the interatomic distance and returns the mapped hamiltonian, as well as some properties of the problem.

def get_qubit_op(d):

    h2 = MoleculeInfo(["H", "H"], 
                        [(0.0, 0.0, 0.0), (0.0, 0.0, d)], 
                        charge=0,
                        multiplicity=1 # 2*spin + 1,
                        )

    driver = PySCFDriver.from_molecule(h2 , basis="sto3g")
    
    problem = driver.run() 
    
    num_particles = problem.num_particles
    
    num_orbitals = int(problem.num_spatial_orbitals)
    
    hamiltonian = problem.hamiltonian
    
    second_q_op = hamiltonian.second_q_op()
    
    mapper = JordanWignerMapper()
    
    converter = QubitConverter(mapper)
    
    qubit_op = converter.convert(second_q_op)

    return qubit_op,problem,converter, num_particles, num_orbitals

# Now we import packages needed for the VQE calculation

from qiskit.algorithms.optimizers import SLSQP, COBYLA, SPSA
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.algorithms.minimum_eigensolvers import VQE

estimator=Estimator()

slsqp = SLSQP()

cobyla = COBYLA()

spsa = SPSA()


# Define a function that calculates the variational ground state energy at a given distance.

def vqe_sol(num_orbitals,num_particles,converter,problem):

    init_state = HartreeFock(num_orbitals,num_particles, converter)
    
    uccsd = UCCSD(num_orbitals, num_particles, converter, initial_state=init_state) 
        
    # two_local = EfficientSU2(num_qubits=4, entanglement='linear', initial_state=init_state)
        
    vqe_slsqp = VQE(estimator, uccsd, slsqp)
    
    vqe_cobyla = VQE(estimator, uccsd, cobyla)
    
    vqe_spsa = VQE(estimator, uccsd, spsa)
    
    vqe_solution_slsqp = vqe_slsqp.compute_minimum_eigenvalue(qubit_op)
    
    vqe_solution_cobyla = vqe_cobyla.compute_minimum_eigenvalue(qubit_op)
    
    vqe_solution_spsa = vqe_spsa.compute_minimum_eigenvalue(qubit_op)
    
    vqe_result_slsqp = problem.interpret(vqe_solution_slsqp).total_energies[0].real
    
    vqe_result_cobyla = problem.interpret(vqe_solution_cobyla).total_energies[0].real
    
    vqe_result_spsa = problem.interpret(vqe_solution_spsa).total_energies[0].real
    
    return vqe_result_slsqp, vqe_result_cobyla, vqe_result_spsa


# Define the arrays to loop over

import numpy as np

distances = np.loadtxt('distances.txt') # Loads data (in a.u.)
distances = distances * 0.529177249 # In Angstrom
exact_ener = np.loadtxt('exact_ener.txt')
vqe_energies_slsqp = []
vqe_energies_cobyla = []
vqe_energies_spsa = []
error_slsqp = []
error_cobyla = []
error_spsa = []


# Start a timer to get the execution time

import timeit

start = timeit.default_timer()

counter = 0;

# Main calculation

for d in distances:
    
    (qubit_op,problem,converter, num_particles, num_orbitals) = get_qubit_op(d)
    
    (vqe_result_slsqp, vqe_result_cobyla, vqe_result_spsa) = vqe_sol(num_orbitals,num_particles,converter,problem)
    
    vqe_energies_slsqp.append(vqe_result_slsqp)        
        
    vqe_energies_cobyla.append(vqe_result_cobyla)   
    
    vqe_energies_spsa.append(vqe_result_spsa)        
    
    print(f"Interatomic Distance: {np.round(d, 2)}",
    f"SLSQP Result: {vqe_result_slsqp:.5f}",
    f" COBYLA Result: {vqe_result_cobyla:.5f}",
    f"Numerical value: {exact_ener[counter]:.5f}")
    
    error_slsqp.append(abs(exact_ener[counter] - vqe_energies_slsqp[counter])) #/exact_ener[counter])
    
    error_cobyla.append(abs(exact_ener[counter] - vqe_energies_cobyla[counter]))
    
    error_spsa.append(abs(exact_ener[counter] - vqe_energies_spsa[counter]))
          
    counter = counter+1
        

print("All energies have been calculated")


stop = timeit.default_timer()

execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds


Interatomic Distance: 0.05 SLSQP Result: 7.39278  COBYLA Result: 7.39278 Numerical value: 7.12722
Interatomic Distance: 0.11 SLSQP Result: 2.42230  COBYLA Result: 2.42230 Numerical value: 2.19780
Interatomic Distance: 0.16 SLSQP Result: 0.80116  COBYLA Result: 0.80116 Numerical value: 0.61924
Interatomic Distance: 0.21 SLSQP Result: 0.02496  COBYLA Result: 0.02496 Numerical value: -0.12023
Interatomic Distance: 0.26 SLSQP Result: -0.41079  COBYLA Result: -0.41079 Numerical value: -0.52664
Interatomic Distance: 0.32 SLSQP Result: -0.67651  COBYLA Result: -0.67651 Numerical value: -0.76964
Interatomic Distance: 0.37 SLSQP Result: -0.84633  COBYLA Result: -0.84633 Numerical value: -0.92203
Interatomic Distance: 0.42 SLSQP Result: -0.95760  COBYLA Result: -0.95760 Numerical value: -1.02006
Interatomic Distance: 0.48 SLSQP Result: -1.03104  COBYLA Result: -1.03104 Numerical value: -1.08364
Interatomic Distance: 0.53 SLSQP Result: -1.07897  COBYLA Result: -1.07897 Numerical value: -1.12454
I